# 地図情報の可視化
### 使用するライブラリ
* folium  
https://python-visualization.github.io/folium/
* plotly.express  
https://plotly.com/python/plotly-express/


### 使用するデータ

* 全国の医療機関の医療提供状況（内閣官房 新型コロナ感染症対策）
  * 掲載サイト：https://corona.go.jp/dashboard
  * Data：covid-19_daily_survey.csv

* Annotated geo-json geometry files for the world  
  * 掲載サイト：https://github.com/johan/world.geo.json
  * Data：countries.geo.json
  * DataURL：https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json

* plotly に同梱されている「世界各国の一人当たりのGDP（2007年）」



## ライブラリの読み込み

In [1]:
# グラフィックに必要なライブラリの読み込み
import folium
from folium.plugins import HeatMap
import plotly.express as px
import json

# データフレームの読み込みに関連するライブラリの読み込み
import pandas as pd
import requests
import io


## folium による地図の描画

### 基本的な地図の描画

In [2]:
# 地図の描画
map0 = folium.Map(
    location = [35,139] ,
    zoom_start = 5.0 ,
    tiles="OpenStreetMap" )
map0

* パラメータ
  * location：地図の中心とする位置（緯度・経度）
  * zoom_start：初期表示倍率
  * tiles：地図のスタイル
  * width：地図の幅
  * height：地図の高さ

* 地図タイルには、以下のようなものが利用できます。
  * OpenStreetMap：デフォルト
  * Stamen Terrain, Stamen Watercolor, Stamen Toner
  * Mapbox Bright：Limited levels of zoom for free tiles
  * Mapbox Control Room：Limited levels of zoom for free tiles



### 位置情報の取得

* 地球上の座標は、経度と緯度で特定できます。
  * 経度：longitude
  * 緯度：latitude

#### 方法１ GoogleMap を使う
* 地図上で目的の場所をダブルクリックすると、マーカーが表示されるとともに住所と緯度・経度が表示されます。

#### 方法２ geojson.io を使う
* 以下のサイトで、地図上にマーカーを置くと、GeoJSON形式のデータが取得できます。  
https://geojson.io

* 参考 九州産業大学 GeoJSONデータ
<pre>
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Point",
        "coordinates": [
          130.44487416744232,
          33.66983627668292
        ]
      }
    }
 </pre>

### 地図上にマーカーを表示

In [3]:
# 九州産業大学の座標を定義
ksu_lat = 33.66983627668292
ksu_lng = 130.44487416744232

# folium.Map 関数でベースとなる地図を作成
map1 = folium.Map(
    location = [ksu_lat, ksu_lng] ,
    zoom_start = 20 )

# folium.Marker 関数でマーカーを定義
folium.Marker(
    location = [ksu_lat, ksu_lng] ,
    popup="九州産業大学"
    ).add_to( map1 )

map1

### 地図上にサークルを表示

In [4]:
# 九州産業大学の座標を定義
ksu_lat = 33.66983627668292
ksu_lng = 130.44487416744232

# folium.Map 関数でベースとなる地図を作成
map2 = folium.Map(
    location = [ksu_lat, ksu_lng] ,
    zoom_start = 15 ,
    tiles="Stamen Terrain" )   # 他に Stamen Toner / Stamen Watercolor

# folium.Circle 関数で円を定義
folium.Circle(
    location = [ksu_lat, ksu_lng] ,
    radius = 200 ,
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
    popup="九州産業大学"
    ).add_to(map2)

map2

### ヒートマップ

* この節では、以下のサイトに掲載されている Covid-19 に対応する全国の医療機関の医療提供状況（内閣官房 新型コロナ感染症対策）をヒートマップとして表示します。  
https://corona.go.jp/dashboard
* プログラムでは、以下の URL からデータを読み込みます。  
https://raw.githubusercontent.com/koichi-inoue/DataScience/main/covid-19_daily_20220823.csv
* データには医療機関の緯度・経度が含まれるので、地図上へのプロットが可能です。

In [5]:
# covid-19_daily_survey.csv（約5MB） の読み込み
df = pd.read_csv('https://raw.githubusercontent.com/koichi-inoue/DataScience/main/covid-19_daily_20220823.csv')
df

,facilityId,facilityName,zipCode,prefName,facilityAddr,facilityTel,latitude,longitude,submitDate,facilityType,ansType,localGovCode,cityName,facilityCode
0,0110515913,医療法人美脳 札幌美しが丘脳神経外科病院,〒004-0834,北海道,札幌市清田区真栄４条５丁目,0115582200,42.982121,141.448688,2022-08-22,入院,未回答,11002,札幌市,110515913
1,0110515913,医療法人美脳 札幌美しが丘脳神経外科病院,〒004-0834,北海道,札幌市清田区真栄４条５丁目,0115582200,42.982121,141.448688,2022-08-22,外来,未回答,11002,札幌市,110515913
2,0110515913,医療法人美脳 札幌美しが丘脳神経外科病院,〒004-0834,北海道,札幌市清田区真栄４条５丁目,0115582200,42.982121,141.448688,2022-08-22,救急,未回答,11002,札幌市,110515913
3,10101075,医療法人 資生会 創成東病院,〒060-0053,北海道,札幌市中央区南３条東３丁目１３番地,0112222117,43.058210,141.361718,2022-08-22,入院,通常,11002,札幌市,110117785
4,10101075,医療法人 資生会 創成東病院,〒060-0053,北海道,札幌市中央区南３条東３丁目１３番地,0112222117,43.058210,141.361718,2022-08-22,外来,通常,11002,札幌市,110117785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24688,14701150,公立久米島病院,〒901-3121,沖縄県,島尻郡久米島町嘉手苅５７２－３,0989855555,26.341873,126.764596,2022-08-22,外来,未回答,473618,久米島町,4718110168
24689,14701150,公立久米島病院,〒901-3121,沖縄県,島尻郡久米島町嘉手苅５７２－３,0989855555,26.341873,126.764596,2022-08-22,救急,未回答,473618,久米島町,4718110168
24690,14701151,医療法人徳洲会 南部徳洲会病院,〒901-0417,沖縄県,島尻郡八重瀬町外間１７１番地１,0989983221,26.178348,127.715544,2022-08-22,入院,通常,473626,八重瀬町,4712311846
24691,14701151,医療法人徳洲会 南部徳洲会病院,〒901-0417,沖縄県,島尻郡八重瀬町外間１７１番地１,0989983221,26.178348,127.715544,2022-08-22,外来,通常,473626,八重瀬町,4712311846


#### ヒートマップの描画

In [8]:
# 東京駅の座標を定義
TOKYO_lat = 35.681382
TOKYO_lng = 139.766083

# folium.Map 関数でベースとなる地図を作成
map3 = folium.Map(
    location = [TOKYO_lat, TOKYO_lng] ,
    zoom_start = 11 ,
    tiles="cartodbpositron" )

# 入院・通常 に該当する病院の座標情報を df_hospital に回収
df_hospital = df.query(
    " facilityType =='入院' & ansType =='未回答'"
    ).loc[:,['latitude','longitude']]

# ヒートマップ用の変数 df_hospital_geolist に df_hospital の値をリスト登録
df_hospital_geolist = df_hospital.values.tolist()

# 各医療機関の座標を半径7 ぼかし5 で map3 に追加
HeatMap( df_hospital_geolist, radius=7, blur=5 ).add_to(map3)

map3


###  GeoJSON データを活用したコロプレス図

#### countries.geo.json の読み込み

In [9]:
# 世界の区画データの読み込み https://github.com/johan/world.geo.json  このデータは PublicDomain で配布されています。
url = requests.get('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json')
text = url.text

# テキストを json 形式のデータとして読み込み
geo_json = json.loads( text)
geo_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'AFG',
   'properties': {'name': 'Afghanistan'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[61.210817, 35.650072],
      [62.230651, 35.270664],
      [62.984662, 35.404041],
      [63.193538, 35.857166],
      [63.982896, 36.007957],
      [64.546479, 36.312073],
      [64.746105, 37.111818],
      [65.588948, 37.305217],
      [65.745631, 37.661164],
      [66.217385, 37.39379],
      [66.518607, 37.362784],
      [67.075782, 37.356144],
      [67.83, 37.144994],
      [68.135562, 37.023115],
      [68.859446, 37.344336],
      [69.196273, 37.151144],
      [69.518785, 37.608997],
      [70.116578, 37.588223],
      [70.270574, 37.735165],
      [70.376304, 38.138396],
      [70.806821, 38.486282],
      [71.348131, 38.258905],
      [71.239404, 37.953265],
      [71.541918, 37.905774],
      [71.448693, 37.065645],
      [71.844638, 36.738171],
      [72.193041, 36.948288],
      [72.63689, 37.047

#### gapminder 2007年のデータ の読み込み

In [10]:
# plotlyに同梱されている gapminder 2007年のデータを読み込み
df_gm = px.data.gapminder().query("year == 2007")
df_gm

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
...,...,...,...,...,...,...,...,...
1655,Vietnam,Asia,2007,74.249,85262356,2441.576404,VNM,704
1667,West Bank and Gaza,Asia,2007,73.422,4018332,3025.349798,PSE,275
1679,"Yemen, Rep.",Asia,2007,62.698,22211743,2280.769906,YEM,887
1691,Zambia,Africa,2007,42.384,11746035,1271.211593,ZMB,894


#### コロプレス図の描画

In [11]:
# 世界地図に gapminder の世界各国の一人当たりのGDP（2007年）データを重ねてコロプレス図を描画

# folium.Map 関数でベースとなる地図を作成
map4 = folium.Map( location = [40,0] , zoom_start = 1.8 )

# folium.Choropleth 関数でコロプレス図を定義
folium.Choropleth(
    geo_data = geo_json,
    data = df_gm,
    fill_opacity = 1,
    line_color = 'white' ,
    nan_fill_color = '#888888' , # 欠損値の色
    columns = ['iso_alpha' , 'gdpPercap'], # 色分けに使うキーとカラム名
    key_on = 'feature.id' , # データに対応した geo.json のキー
    fill_color = 'OrRd',
).add_to(map4)

map4


* fill_color
  * Default：Blue
  * Others：BuGn, BuPu, GnBu, OrRd, PuBu, PuBuGn, PuRd, RdPu, YlGn, YlGnBu, YlOrBr, YlOrRd

## plotly.express による地球儀

In [12]:
# 地球儀に国名をドットで表示
df_gm = px.data.gapminder().query("year == 2007")
fig0 = px.scatter_geo( df_gm, locations="iso_alpha", color="continent", projection="orthographic")
fig0.show()

In [13]:
# 地球儀に gapminder の各国の人口（2007年）を重ねて表示
df_gm = px.data.gapminder().query("year == 2007")
fig1 = px.scatter_geo( df_gm, locations='iso_alpha', color='continent', hover_name='country', size='pop', projection='orthographic' )
fig1.show()